# What are `TargetPixelFile` objects?

Target Pixel Files (TPFs) are a file common to Kepler/K2 and the TESS mission. They contain movies of the pixel data centered on a single target star.

TPFs can be thought of as stacks of images, with one image for every timestamp the telescope took data. Each timestamp is referred to as a **cadence**. These images are cut out 'postage stamps' of the full observation to make them easier to work with. 

TPFs are given in FITS files, which you can read more about [here](https://fits.gsfc.nasa.gov/fits_primer.html). *Lightkurve* includes tools for you to work directly with these files easily and intuitively.

In this tutorial we'll cover the basics of working with TPFs. In *Lightkurve* there are classes to work with each mission. For example [`KeplerTargetPixelFile`](https://docs.lightkurve.org/api/lightkurve.targetpixelfile.KeplerTargetPixelFile.html#lightkurve.targetpixelfile.KeplerTargetPixelFile) deals with data from the Kepler (and K2) mission. [`TessTargetPixelFile`](https://docs.lightkurve.org/api/lightkurve.lightcurvefile.TessLightCurveFile.html#lightkurve.lightcurvefile.TessLightCurveFile) deals with data from the TESS mission. We'll use a Kepler TPF as an example.

To load a `KeplerTargetPixelFile` from a local path or remote url, simply create a new object using the location of the file as the parameter. All Kepler Target  pixel files can be found at this [parent directory](https://archive.stsci.edu/pub/kepler/target_pixel_files/) which contains information for each object observed.

In [ ]:
import numpy as np
import pandas as pd

from lightkurve import KeplerTargetPixelFile
tpf = KeplerTargetPixelFile("https://archive.stsci.edu/pub/kepler/target_pixel_files/0069/006922244/kplr006922244-2010078095331_lpd-targ.fits.gz")

You can also search for the url automatically using the [`search_targetpixelfile()`](https://docs.lightkurve.org/api/lightkurve.search.search_targetpixelfile.html#lightkurve.search.search_targetpixelfile) function. This will search for the right file in the [MAST data archive](https://archive.stsci.edu/kepler/) which holds all of the Kepler and K2 data.
In this case we want the Target Pixel File with Kepler ID 6922244 for Quarter 4 (Kepler's observations were split into quarters of a year):

In [ ]:
from lightkurve import search_targetpixelfile
tpf = search_targetpixelfile('KIC 6922244', quarter=4).download()

You can also pass the name of the target or its astronomical coordinates as a parameter to `search_targetpixelfile()`.

The above code has created a variable named `tpf` which is a Python object of type `KeplerTargetPixelFile`:

In [ ]:
tpf

We can access lots of meta data using this object in a simple way. For example, we can find the mission name, and the quarter that the data was taken in by typing the following:

In [ ]:
tpf.mission

In [ ]:
tpf.quarter

You can find the full list of properties in the [API documentation](http://lightkurve.keplerscience.org/api/targetpixelfile.html) on this object.

The most interesting data in a `KeplerTargetPixelFile` object are the `flux` (units electrons/second) and `time` values which give access to the brightness of the observed target over time. You can access the timestamps of the observations using the `time` property:

In [ ]:
times = pd.DataFrame(data={'Time (BKJD)': tpf.time})
print(times)

By default, `time` is in the Kepler-specific *Barycentric Kepler Julian Day* format (BKJD - see Section 6.2.2 of the [Kepler manual](https://archive.stsci.edu/kepler/manuals/Data_Characteristics_Handbook_20110201.pdf)). You can easily convert this into [AstroPy Time objects](http://docs.astropy.org/en/stable/time/) using the `astropy_time` property:

In [ ]:
tpf.astropy_time
times_jd = pd.DataFrame(data={'Time (JD)': tpf.astropy_time})
times_jd

In turn, this gives you access to human-readable ISO timestamps using the `astropy_time.iso` property:

In [ ]:
tpf.astropy_time.iso
times_iso = pd.DataFrame(data={'Time (YYYY-MM-DD HH:MM:SS)': tpf.astropy_time.iso})
times_iso

**Beware:** these timestamps are in the Solar System Barycentric frame (TDB) and do not include corrections for light travel time or leap seconds.  To use a different time scale, such as the Earth-centered UTC system, you can use [AstroPy's time scale conversion features](http://docs.astropy.org/en/stable/time/#time-scale).  For example: 

In [ ]:
tpf.astropy_time.utc.iso
times_utc = pd.DataFrame(data={'Time (YYYY-MM-DD HH:MM:SS)': tpf.astropy_time.utc.iso})
times_utc

Next, let's look at the actual image data, which is available via the `flux` property:

In [ ]:
tpf.flux.shape

The `flux` data is a 4116x5x5 array in units electrons/second. The first axis is the time axis, and the images themselves are 5 pixels by 5 pixels. You can use the `plot` method on the `KeplerTargetPixelFile` object to view the data. (By default, this will show just one cadence of the data. But you can pass the cadence you want to look at to the `frame` keyword if you would like to check a particular flux point for thruster firings, cosmic rays or asteroids.)

In [ ]:
%matplotlib inline
tpf.plot(frame=0);

The values shown in this image are also directly accessible as an array:

In [ ]:
tpf.flux[0]
flux_0 = pd.DataFrame(data={'0': tpf.flux[0][0], '1': tpf.flux[0][1], '2': tpf.flux[0][2], '3': tpf.flux[0][3], '4': tpf.flux[0][4]})
flux_0

You can use normal `numpy` methods on these to find the shape, mean etc!

We can now turn this Target Pixel File into a light curve, with a single flux value for every time value. Each of the pixels are 4 arcseconds across. The point spread function (PSF) of the telescope causes the light from the star fall onto several different pixels, which can be seen in the image above. For more information about the *Kepler* PSF please see the [Kepler archive manual](http://archive.stsci.edu/kepler/manuals/archive_manual.pdf) and the paper by [Bryson et al., 2010](https://ui.adsabs.harvard.edu/abs/2010ApJ...713L..97B/abstract).

Because of the spreading of the flux, we have to sum up many pixels to collect all the light from the source. To do this we sum up all the pixels in an **aperture**. An aperture is a pixel mask, where we take only the pixels related to the target. 

The *Kepler* pipeline adds an aperture mask to each target pixel file. This aperture determines which pixels are summed to create a 1-D light curve of the target. There are some science cases where you might want to create a different aperture. For example, there may be a nearby contaminant or you may want to measure the background. 

The standard pipeline aperture is easily accessed in a `KeplerTargetPixelFile` object using [`tpf.[pipeline_mask`](https://docs.lightkurve.org/api/lightkurve.targetpixelfile.KeplerTargetPixelFile.html#lightkurve.targetpixelfile.KeplerTargetPixelFile.pipeline_mask), which is a boolean array:

In [ ]:
tpf.pipeline_mask

We can also plot this aperture over the target pixel file above to see if the flux of the star is all contained within the aperture.

In [ ]:
tpf.plot(aperture_mask=tpf.pipeline_mask);

The shaded red region is the aperture and it looks to cover all desired pixels.

Now that we have the aperture we can create a Simple Aperture Photometry light curve in the [next tutorial](https://docs.lightkurve.org/tutorials/01-what-are-lightcurves.html).

Finally, note that you can inspect all the raw metadata of the target by taking a look at the 'header' of the FITS file, which contains information about the data set:

In [ ]:
tpf.header[:]

We can look at the values in the second extension of the fits file by accessing the AstroPy FITS `HDUList` object. For example, to look at all the column titles:

In [ ]:
tpf.hdu[1].header#['TTYPE*']

This is a lot of information to process and if you would prefer to only look at certain items you can specify a string such as 'TYPE' for instance:

In [ ]:
tpf.hdu[1].header['TTYPE*']

You can find more information on FITS file handeling [here](https://docs.astropy.org/en/stable/io/fits/)